# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
weather_file = "C:\Data\weatherpy_city_data.csv"
weather_df = pd.read_csv(weather_file)

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,75.45,80,78,15.68,PF,1635451734
1,Hobart,-42.8794,147.3294,48.06,93,90,3.00,AU,1635451701
2,Pitimbu,-7.4706,-34.8086,81.19,66,87,13.91,BR,1635451735
3,Atuona,-9.8000,-139.0333,78.10,71,0,16.93,PF,1635451735
4,Arraial do Cabo,-22.9661,-42.0278,73.26,94,75,12.66,BR,1635451456


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
gmaps.configure(api_key=g_key)

weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

locations = weather_df[["Lat", "Lng"]].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
narrowed_weather_df = weather_df

# A max temperature lower than 80 degrees but higher than 70.
narrowed_weather_df = narrowed_weather_df.loc[(narrowed_weather_df["Max Temp"] < 80) & (narrowed_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
narrowed_weather_df = narrowed_weather_df.loc[narrowed_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
narrowed_weather_df = narrowed_weather_df.loc[narrowed_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
narrowed_weather_df = narrowed_weather_df.dropna()

narrowed_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,Sur,22.5667,59.5289,77.95,66,0,3.02,OM,1635451753
169,San Quintín,30.4833,-115.9500,76.06,36,0,6.73,MX,1635451790
202,Souillac,-20.5167,57.5167,75.36,78,0,5.53,MU,1635451801
237,Ormara,25.2088,64.6357,78.73,62,0,7.16,PK,1635451818
267,Bosaso,11.2842,49.1816,79.72,62,0,6.22,SO,1635451827
554,Noumea,-22.2763,166.4572,75.00,99,0,8.55,NC,1635451811


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
hotelmap_df = narrowed_weather_df

# dictionary
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotelmap_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotelmap_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing... skipping.")
        
    print("------------")

Retrieving Results for Index 54: Sur.
Closest hotel is Sur Plaza Hotel.
------------
Retrieving Results for Index 169: San Quintín.
Closest hotel is Don Eddie's Sport Fishing Center.
------------
Retrieving Results for Index 202: Souillac.
Closest hotel is Shanti Maurice Resort & Spa.
------------
Retrieving Results for Index 237: Ormara.
Closest hotel is Crystal Beach and Camping Resort Ormara.
------------
Retrieving Results for Index 267: Bosaso.
Closest hotel is Red Sea Hotel.
------------
Retrieving Results for Index 554: Noumea.
Closest hotel is Hôtel Le Lagon.
------------


In [20]:
hotelmap_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
54,Sur,22.5667,59.5289,77.95,66,0,3.02,OM,1635451753,Sur Plaza Hotel
169,San Quintín,30.4833,-115.9500,76.06,36,0,6.73,MX,1635451790,Don Eddie's Sport Fishing Center
202,Souillac,-20.5167,57.5167,75.36,78,0,5.53,MU,1635451801,Shanti Maurice Resort & Spa
237,Ormara,25.2088,64.6357,78.73,62,0,7.16,PK,1635451818,Crystal Beach and Camping Resort Ormara
267,Bosaso,11.2842,49.1816,79.72,62,0,6.22,SO,1635451827,Red Sea Hotel
554,Noumea,-22.2763,166.4572,75.00,99,0,8.55,NC,1635451811,Hôtel Le Lagon


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotelmap_df.iterrows()]
locations = hotelmap_df[["Lat", "Lng"]]

In [31]:
# Add marker layer ontop of heat map
hotel_info = [info_box_template.format(**row) for index, row in hotelmap_df.iterrows()]
locations = hotelmap_df[["Lat", "Lng"]]

# Display figure
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))